# Lab | Data Cleaning and Formatting

In this lab, we will be working with the customer data from an insurance company, which can be found in the CSV file located at the following link: https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv


# Challenge 1: Data Cleaning and Formatting

## Exercise 1: Cleaning Column Names

To ensure consistency and ease of use, standardize the column names of the dataframe. Start by taking a first look at the dataframe and identifying any column names that need to be modified. Use appropriate naming conventions and make sure that column names are descriptive and informative.

*Hint*:
- *Column names should be in lower case*
- *White spaces in column names should be replaced by `_`*
- *`st` could be replaced for `state`*

In [459]:
import pandas as pd
import numpy as np
ins_df=pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')


ins_df.columns=[x.lower() for x in ins_df.columns]
ins_df.columns=map(lambda x:x.replace(" ","_"),ins_df.columns)
ins_df.rename(columns={"st":"state"},inplace=True)

ins_df.head()

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323


## Exercise 2: Cleaning invalid Values

The dataset contains columns with inconsistent and incorrect values that could affect the accuracy of our analysis. Therefore, we need to clean these columns to ensure that they only contain valid data.

Note that this exercise will focus only on cleaning inconsistent values and will not involve handling null values (NaN or None).

*Hint*:
- *Gender column contains various inconsistent values such as "F", "M", "Femal", "Male", "female", which need to be standardized, for example, to "M" and "F".*
- *State abbreviations be can replaced with its full name, for example "AZ": "Arizona", "Cali": "California", "WA": "Washington"*
- *In education, "Bachelors" could be replaced by "Bachelor"*
- *In Customer Lifetime Value, delete the `%` character*
- *In vehicle class, "Sports Car", "Luxury SUV" and "Luxury Car" could be replaced by "Luxury"*

In [462]:
ins_df["gender"]=np.where(ins_df["gender"] == 'female', 'F', ins_df["gender"])
ins_df["gender"]=np.where(ins_df["gender"] == 'Femal', 'F', ins_df["gender"])
ins_df["gender"]=np.where(ins_df["gender"] == 'Male', 'M', ins_df["gender"])

ins_df["state"] = np.where(ins_df["state"] == 'WA', 'Washington', ins_df["state"])
ins_df["state"] = np.where(ins_df["state"] == 'AZ', 'Arizona', ins_df["state"])
ins_df["state"] = np.where(ins_df["state"] == 'Cali', 'California', ins_df["state"])

ins_df["education"] = np.where(ins_df["education"] == 'Bachelors', 'Bachelor', ins_df["education"])

ins_df["customer_lifetime_value"]=ins_df["customer_lifetime_value"].str.replace('%', '')


ins_df["vehicle_class"]=ins_df["vehicle_class"].replace(('Sports Car','Luxury SUV','Luxury Car'),'Luxury')

ins_df.nunique()
ins_df["customer_lifetime_value"]

0              NaN
1        697953.59
2       1288743.17
3        764586.18
4        536307.65
           ...    
4003           NaN
4004           NaN
4005           NaN
4006           NaN
4007           NaN
Name: customer_lifetime_value, Length: 4008, dtype: object

## Exercise 3: Formatting data types

The data types of many columns in the dataset appear to be incorrect. This could impact the accuracy of our analysis. To ensure accurate analysis, we need to correct the data types of these columns. Please update the data types of the columns as appropriate.

It is important to note that this exercise does not involve handling null values (NaN or None).

*Hint*:
- *Customer lifetime value should be numeric*
- *Number of open complaints has an incorrect format. Look at the different values it takes with `unique()` and take the middle value. As an example, 1/5/00 should be 5. Number of open complaints is a string - remember you can use `split()` to deal with it and take the number you need. Finally, since it should be numeric, cast the column to be in its proper type.*

In [471]:
ins_df["customer_lifetime_value"]=ins_df["customer_lifetime_value"].astype(float)

ins_df["number_of_open_complaints"]=ins_df["number_of_open_complaints"].astype(str)

ins_df['number_of_open_complaints']=ins_df['number_of_open_complaints'].apply(lambda x:x.split("/")[1] if pd.notna(x) and len(x.split("/")) > 1 else np.nan) 

ins_df['number_of_open_complaints'].apply(lambda x:int(x) if pd.notna(x) else np.nan) 

ins_df.dtypes
ins_df.head()

ins_df['number_of_open_complaints'].value_counts()



Series([], Name: count, dtype: int64)


## Exercise 4: Dealing with Null values

Identify any columns with null or missing values. Identify how many null values each column has. You can use the `isnull()` function in pandas to find columns with null values.

Decide on a strategy for handling the null values. There are several options, including:

- Drop the rows or columns with null values
- Fill the null values with a specific value (such as the column mean or median for numerical variables, and mode for categorical variables)
- Fill the null values with the previous or next value in the column
- Fill the null values based on a more complex algorithm or model (note: we haven't covered this yet)

Implement your chosen strategy to handle the null values. You can use the `fillna()` function in pandas to fill null values or `dropna()` function to drop null values.

Verify that your strategy has successfully handled the null values. You can use the `isnull()` function again to check if there are still null values in the dataset.

Remember to document your process and explain your reasoning for choosing a particular strategy for handling null values.

After formatting data types, as a last step, convert all the numeric variables to integers.

In [473]:
print(ins_df.isna().sum())
ins_df.shape

#removing rows commpletely empty, if any:
ins_df=ins_df.dropna(how='all')

#Now, we only have missing values in columns customer_lifetime_value and education.
#We will use the average of customer_lifetime_value to fill in the 3 missing values.
#ins_df['customer_lifetime_value']=ins_df['customer_lifetime_value'].fillna(ins_df['customer_lifetime_value'].mean())

#We will replace the missing values in gender to be "X" for unknown:
ins_df['gender']=ins_df['gender'].fillna('X')

ins_df.isna().sum()
print(ins_df.head())

#Converting into interger:
ins_df['income']=ins_df['income'].astype(int)
ins_df['monthly_premium_auto']=ins_df['monthly_premium_auto'].astype(int)
ins_df['total_claim_amount']=ins_df['total_claim_amount'].astype(int)


ins_df.dtypes

customer                        0
state                           0
gender                          0
education                       0
customer_lifetime_value         3
income                          0
monthly_premium_auto            0
number_of_open_complaints    1071
policy_type                     0
vehicle_class                   0
total_claim_amount              0
dtype: int64
  customer       state gender             education  customer_lifetime_value  \
0  RB50392  Washington      X                Master                      NaN   
1  QZ44356     Arizona      F              Bachelor                697953.59   
2  AI49188      Nevada      F              Bachelor               1288743.17   
3  WW63253  California      M              Bachelor                764586.18   
4  GA49547  Washington      M  High School or Below                536307.65   

   income  monthly_premium_auto  number_of_open_complaints     policy_type  \
0       0                  1000                       

customer                      object
state                         object
gender                        object
education                     object
customer_lifetime_value      float64
income                         int64
monthly_premium_auto           int64
number_of_open_complaints    float64
policy_type                   object
vehicle_class                 object
total_claim_amount             int64
dtype: object

## Exercise 5: Dealing with duplicates

Use the `.duplicated()` method to identify any duplicate rows in the dataframe.

Decide on a strategy for handling the duplicates. Options include:
- Dropping all duplicate rows
- Keeping only the first occurrence of each duplicated row
- Keeping only the last occurrence of each duplicated row
- Dropping duplicates based on a subset of columns
- Dropping duplicates based on a specific column

Implement your chosen strategy using the `drop_duplicates()` function.

Verify that your strategy has successfully handled the duplicates by checking for duplicates again using `.duplicated()`.

Remember to document your process and explain your reasoning for choosing a particular strategy for handling duplicates.

Save the cleaned dataset to a new CSV file.

*Hint*: *after dropping duplicates, reset the index to ensure consistency*.

In [493]:

ins_df_no_duplicates=ins_df.copy()
print(ins_df_no_duplicates.head())
ins_df_no_duplicates.shape

ins_df_no_duplicates.drop_duplicates()
ins_df_no_duplicates.duplicated(subset=['customer']).sum()
#There is no duplicate row. There is no duplicate in column customer




  customer       state gender             education  customer_lifetime_value  \
0  RB50392  Washington      X                Master                      NaN   
1  QZ44356     Arizona      F              Bachelor                697953.59   
2  AI49188      Nevada      F              Bachelor               1288743.17   
3  WW63253  California      M              Bachelor                764586.18   
4  GA49547  Washington      M  High School or Below                536307.65   

   income  monthly_premium_auto  number_of_open_complaints     policy_type  \
0       0                  1000                        NaN   Personal Auto   
1       0                    94                        NaN   Personal Auto   
2   48767                   108                        NaN   Personal Auto   
3       0                   106                        NaN  Corporate Auto   
4   36357                    68                        NaN   Personal Auto   

   vehicle_class  total_claim_amount  
0  Four-Doo

0

# Bonus: Challenge 2: creating functions on a separate `py` file

Put all the data cleaning and formatting steps into functions, and create a main function that performs all the cleaning and formatting.

Write these functions in separate .py file(s). By putting these steps into functions, we can make the code more modular and easier to maintain.

*Hint: autoreload module is a utility module in Python that allows you to automatically reload modules in the current session when changes are made to the source code. This can be useful in situations where you are actively developing code and want to see the effects of changes you make without having to constantly restart the Python interpreter or Jupyter Notebook kernel.*

In [495]:
def clean_dot(x):
    return x.replace(".","")

def clean_underscore(x):
    return x.replace("_","")

def clean_comma(x):
    return x.replace(",","")

def clean_percentage(x):
    return x.replace("%","")

def clean_whitespace(x):
    return x.replace(" ","")

def clean_df(df, columns):
    for col in columns:
        df[col] = df[col].apply(clean_dot)
        df[col] = df[col].apply(clean_underscore)
        df[col] = df[col].apply(clean_comma)
        df[col] = df[col].apply(clean_percentage)
        df[col] = df[col].apply(clean_whitespace)
        return df

clean_df(ins_df_no_duplicates,['education'])

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,RB50392,Washington,X,Master,NaN,0,1000,NaN,Personal Auto,Four-Door Car,2
1,QZ44356,Arizona,F,Bachelor,697953.59,0,94,NaN,Personal Auto,Four-Door Car,1131
2,AI49188,Nevada,F,Bachelor,1288743.17,48767,108,NaN,Personal Auto,Two-Door Car,566
3,WW63253,California,M,Bachelor,764586.18,0,106,NaN,Corporate Auto,SUV,529
4,GA49547,Washington,M,HighSchoolorBelow,536307.65,36357,68,NaN,Personal Auto,Four-Door Car,17
...,...,...,...,...,...,...,...,...,...,...,...
1066,TM65736,Oregon,M,Master,305955.03,38644,78,NaN,Personal Auto,Four-Door Car,361
1067,VJ51327,California,F,HighSchoolorBelow,2031499.76,63209,102,NaN,Personal Auto,SUV,207
1068,GS98873,Arizona,F,Bachelor,323912.47,16061,88,NaN,Personal Auto,Four-Door Car,633
1069,CW49887,California,F,Master,462680.11,79487,114,NaN,Special Auto,SUV,547


# Bonus: Challenge 3: Analyzing Clean and Formated Data

You have been tasked with analyzing the data to identify potential areas for improving customer retention and profitability. Your goal is to identify customers with a high policy claim amount and a low customer lifetime value.

In the Pandas Lab, we only looked at high policy claim amounts because we couldn't look into low customer lifetime values. If we had tried to work with that column, we wouldn't have been able to because customer lifetime value wasn't clean and in its proper format. So after cleaning and formatting the data, let's get some more interesting insights!

Instructions:

- Review the statistics again for total claim amount and customer lifetime value to gain an understanding of the data.
- To identify potential areas for improving customer retention and profitability, we want to focus on customers with a high policy claim amount and a low customer lifetime value. Consider customers with a high policy claim amount to be those in the top 25% of the total claim amount, and clients with a low customer lifetime value to be those in the bottom 25% of the customer lifetime value. Create a pandas DataFrame object that contains information about customers with a policy claim amount greater than the 75th percentile and a customer lifetime value in the bottom 25th percentile.
- Use DataFrame methods to calculate summary statistics about the high policy claim amount and low customer lifetime value data. To do so, select both columns of the dataframe simultaneously and pass it to the `.describe()` method. This will give you descriptive statistics, such as mean, median, standard deviation, minimum and maximum values for both columns at the same time, allowing you to compare and analyze their characteristics.

In [477]:
claim_amount_75percentile=ins_df['total_claim_amount'].quantile(0.75)
print(f"75th percentile (quantile 0.75): {claim_amount_75percentile}")

lifetime_value_25percentile=ins_df['customer_lifetime_value'].quantile(0.25)
print(f"25th percentile (quantile 0.25): {lifetime_value_25percentile}")

75th percentile (quantile 0.75): 532.0
25th percentile (quantile 0.25): 403407.995


In [489]:
claim_amount_top25_df=ins_df[ins_df['total_claim_amount']>532.8]
claim_amount_top25_df

lifetime_value_bottom25_df=ins_df[ins_df['customer_lifetime_value']<403407.995]
lifetime_value_bottom25_df

filtered_df=ins_df[(ins_df['total_claim_amount']>532.8) & (ins_df['customer_lifetime_value']<403407.995)]

filtered_df[['total_claim_amount','customer_lifetime_value']].describe()


,total_claim_amount,customer_lifetime_value
count,40.000000,40.000000
mean,723.300000,341577.910000
std,149.216449,56921.472027
min,537.000000,228759.690000
25%,632.750000,304220.875000
50%,685.000000,365995.370000
75%,799.000000,387364.700000
max,1185.000000,402381.440000
